# Pré-Processamento dos Dados

Nesse notebook será feito um pré-processamento dos acórdãos classificados previamente para que os dados possam ser usados no treinamento e construção dos modelos e na avaliação das métricas.

O pré-processamento consiste de duas etapas:
- A primeira é uma etapa de mapeamento. A resposta esperada pelos modelos seria o ramo ao qual o acórdão pertence. Dessa forma, padronizar esse dado resulta em torná-lo mais inteligível para o BERT. Portanto, uma enum foi criada para representar cada um dos ramos do direito com quais o modelo irá trabalhar.
- A segunda etapa consiste da separação dos dados. Para que os modelos possam ser construídos e treinados são necessários conjuntos de dados de treinamento e de validação, enquanto que para que sejam feitas avaliações dos modelos é necessário um conjunto de dados de teste. Assim, é feita uma separação dos dados em três conjuntos de dados: um de treinamento, um de validação e um de teste nas proporções de 70%, 15% e 15%, respectivamente.

In [ ]:
# Imports

from google.colab import drive
import pandas as pd
from enum import Enum
from sklearn.model_selection import train_test_split

In [ ]:
# Constants

CONSTANTS = {
    'SPREADSHEET_PATH': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Acordaos_Final_Full_Info_Dataset.csv',
    'TRAINING_PATH': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Train_Dataset.csv',
    'VALIDATION_PATH': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Validation_Dataset.csv',
    'TEST_PATH': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Test_Dataset.csv'
}

In [ ]:
# Mounting Google Drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Definition of mapping from law branch name to a numeric identifier

class LawBranch(Enum):
    """Mapping to a Law Branch and an identification. The enum also stores
    the law branch name in a free text form.
    """

    Penal = (0, "Direito Penal (Direito Processual Penal)")
    Administrativo = (1, "Direito Administrativo (Licitações, Contratos Administrativos, Servidores, Desapropriação, Tribunal de Contas, Improbidade, etc.)")
    Tributario = (2, "Direito Tributário/Direito Financeiro")
    Civil = (3, "Direito Civil (Direito Comercial/Direito de Família)")
    Previdenciario = (4, "Direito Previdenciário")
    Trabalho = (5, "Direito do Trabalho")
    Processual_Civil = (6, "Direito Processual Civil")
    Eleitoral = (7, "Direito Eleitoral")
    Consumidor = (8, "Direito do Consumidor")
    Internacional = (9, "Direito Internacional (Público ou Privado)")
    Militar = (10, "Direito Militar")
    Economico = (11, "Direito Econômico (Direito concorrencial e Agências Reguladoras Setoriais, Intervenção no Domínio Econômico)")
    Ambiental = (12, "Direito Ambiental")

    def get_identifier(self) -> int:
        """Retrieves the identifier number for this instance of LawBranch.

        Returns:
            int: identifier of this instance of LawBranch
        """
        return self.value[0]

    @staticmethod
    def from_branch_name(branch_name: str) -> 'LawBranch':
        """Retrieves an instance of LawBranch given the branch name in a free
        text form.

        Args:
            branch_name (str): the branch name as string value.

        Returns:
            LawBranch: instance of LawBranch retrieved according to the given
            branch name.

        Raises:
            ValueError: when there is no match with given branch name and an
            instance of LawBranch
        """
        for law_branch in LawBranch:
            if law_branch.value[1] == branch_name:
                return law_branch

        raise ValueError("Impossible to find an instance of LawBranch from branch_name={branch_name}".format(branch_name=branch_name))

In [ ]:
# Read the full dataset from .csv file

documents = pd.read_csv(CONSTANTS['SPREADSHEET_PATH'])
documents['ementa'] = documents['ementa'].apply(str)
documents

,Unnamed: 0,cod_acordao,ramo,tipo_acordao,cabecalho,ementa,decisao,indexacao,somente_ementa,indicacao_exclusiva_ementa_voto,expressoes_chave
0,0,EXT 1259,Direito Penal (Direito Processual Penal),EXT,Ext 1259 / DF - DISTRITO FEDERAL EXTRADIÇÃO Re...,EMENTA Extradição instrutória. Duplicidade do ...,"A Turma deferiu a extradição, nos termos do vo...","['INEXISTÊNCIA', 'CONOTAÇÃO', 'CRIME POLÍTICO'...",Sim,Sim,prática delitiva
1,1,RCL 1601,Direito Previdenciário,RCL,Rcl 1601 / SE - SERGIPE RECLAMAÇÃO Relator(a):...,"Reclamação. Tutela antecipada. Decisão que, an...",O Tribunal julgou improcedente a reclamação. V...,"['(CÍVEL)', 'DESCABIMENTO', 'RECLAMAÇÃO', 'INS...",Sim,Sim,tutela antecipada
2,2,MS 23855,Direito Civil (Direito Comercial/Direito de Fa...,MS,MS 23855 / MS - MATO GROSSO DO SUL MANDADO DE ...,"EMENTA: Mandado de segurança, contra ato do Pr...","O Tribunal, por unanimidade, concedeu a segura...","['(CÍVEL)', 'NULIDADE', 'DECRETO PRESIDENCIAL'...",Sim,Não,"""imóvel rural""; ""imóvel""; ""proprietário"""
3,3,EXT 1370,Direito Internacional (Público ou Privado),EXT,Ext 1370 / DF - DISTRITO FEDERAL EXTRADIÇÃO Re...,Ementa: Direito Internacional Público. Extradi...,"A Turma deferiu o pedido de extradição, nos te...",['VIDE EMENTA'],Sim,Sim,direito público internacional
4,4,AO 1833,"Direito Administrativo (Licitações, Contratos ...",AO,AO 1833 / AC - ACRE AÇÃO ORIGINÁRIA Relator(a)...,Ementa: AÇÃO ORIGINÁRIA. APELAÇÃO EM AÇÃO CIVI...,"A Turma, por unanimidade, deu parcial provimen...","['NOTIFICAÇÃO PRÉVIA', 'REDAÇÃO ORIGINAL', 'LE...",Sim,Sim,uso irregular de verbas públicas
...,...,...,...,...,...,...,...,...,...,...,...
5519,5767,RE 228220,Direito do Trabalho,RE,RE 228220 / MG - MINAS GERAIS RECURSO EXTRAORD...,EMENTA: TRABALHISTA. REDE FERROVIÁRIA FEDERAL ...,A Turma não conheceu do recurso extraordinário...,"['INADMISSIBILIDADE', 'RECURSO EXTRAORDINÁRIO'...",Sim,Sim,trabalhista
5520,5768,HC 111586,Direito Penal (Direito Processual Penal),HC,HC 111586 / SP - SÃO PAULO HABEAS CORPUS Relat...,EMENTA Habeas corpus. Constitucional. Processu...,"Por maioria de votos, a Turma concedeu, em par...","['ENTENDIMENTO', 'PLENÁRIO', 'SUPREMO TRIBUNAL...",Sim,Sim,"""habeas corpus""; ""tráfico de entorpecentes"" ; ..."
5521,5769,RHC 118977,Direito Penal (Direito Processual Penal),RHC,RHC 118977 / MS - MATO GROSSO DO SUL RECURSO O...,EMENTA Recurso ordinário em habeas corpus. Pro...,A Turma negou provimento ao recurso ordinário ...,"['IMPOSSIBILIDADE', 'CONSIDERAÇÃO', 'CONDENAÇÃ...",Sim,Sim,"""Constrangimento ilegal""; ""habeas corpus""; ""re..."
5522,5770,HC 126665,Direito Penal (Direito Processual Penal),HC,HC 126665 / SP - SÃO PAULO HABEAS CORPUS Relat...,Ementa: HABEAS CORPUS CONTRA DECISÃO MONOCRÁTI...,"Por maioria de votos, a Turma não admitiu a im...","['VIDE EMENTA', 'FUNDAMENTAÇÃO COMPLEMENTAR', ...",Sim,Sim,"""Furto e receptação""; ""Prisão preventiva""; ""ga..."


In [ ]:
# Convert the values in branch law column in the dataset into numerical identifiers

documents['ramo'] = documents['ramo'].apply(lambda law_branch_name: LawBranch.from_branch_name(law_branch_name).get_identifier())
documents.head()

,Unnamed: 0,cod_acordao,ramo,tipo_acordao,cabecalho,ementa,decisao,indexacao,somente_ementa,indicacao_exclusiva_ementa_voto,expressoes_chave
0,0,EXT 1259,0,EXT,Ext 1259 / DF - DISTRITO FEDERAL EXTRADIÇÃO Re...,EMENTA Extradição instrutória. Duplicidade do ...,"A Turma deferiu a extradição, nos termos do vo...","['INEXISTÊNCIA', 'CONOTAÇÃO', 'CRIME POLÍTICO'...",Sim,Sim,prática delitiva
1,1,RCL 1601,4,RCL,Rcl 1601 / SE - SERGIPE RECLAMAÇÃO Relator(a):...,"Reclamação. Tutela antecipada. Decisão que, an...",O Tribunal julgou improcedente a reclamação. V...,"['(CÍVEL)', 'DESCABIMENTO', 'RECLAMAÇÃO', 'INS...",Sim,Sim,tutela antecipada
2,2,MS 23855,3,MS,MS 23855 / MS - MATO GROSSO DO SUL MANDADO DE ...,"EMENTA: Mandado de segurança, contra ato do Pr...","O Tribunal, por unanimidade, concedeu a segura...","['(CÍVEL)', 'NULIDADE', 'DECRETO PRESIDENCIAL'...",Sim,Não,"""imóvel rural""; ""imóvel""; ""proprietário"""
3,3,EXT 1370,9,EXT,Ext 1370 / DF - DISTRITO FEDERAL EXTRADIÇÃO Re...,Ementa: Direito Internacional Público. Extradi...,"A Turma deferiu o pedido de extradição, nos te...",['VIDE EMENTA'],Sim,Sim,direito público internacional
4,4,AO 1833,1,AO,AO 1833 / AC - ACRE AÇÃO ORIGINÁRIA Relator(a)...,Ementa: AÇÃO ORIGINÁRIA. APELAÇÃO EM AÇÃO CIVI...,"A Turma, por unanimidade, deu parcial provimen...","['NOTIFICAÇÃO PRÉVIA', 'REDAÇÃO ORIGINAL', 'LE...",Sim,Sim,uso irregular de verbas públicas


In [ ]:
# Split documents dataset into train, test and validation datasets

train_dataset, test_dataset = train_test_split(documents, test_size=0.3)
val_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5)

In [ ]:
# Remove unnecessary column from datasets

train_dataset = train_dataset.drop(columns=['Unnamed: 0'])
test_dataset = test_dataset.drop(columns=['Unnamed: 0'])
val_dataset = val_dataset.drop(columns=['Unnamed: 0'])

In [ ]:
# Save datasets into respectives .csv files

train_dataset.to_csv(CONSTANTS['TRAINING_PATH'])
test_dataset.to_csv(CONSTANTS['TEST_PATH'])
val_dataset.to_csv(CONSTANTS['VALIDATION_PATH'])